<a href="https://colab.research.google.com/github/gaurang2112/first_CNN/blob/master/VGG_KN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Kaggle library
!pip install -q kaggle

In [2]:
from google.colab import files

files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gaurang2112","key":"52205ecfe9ebc6c75c41b3db40dd97f8"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d alessiocorrado99/animals10   

 98% 576M/586M [00:24<00:00, 32.2MB/s]
100% 586M/586M [00:24<00:00, 25.2MB/s]


In [5]:
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: raw-img/ragno/OIP-MVdIFDAFpNyK_XCpwkDIWgHaFj.jpeg  
  inflating: raw-img/ragno/OIP-MYY_8k5AHqMJwTCNXfH6rwHaFj.jpeg  
  inflating: raw-img/ragno/OIP-MZEVXuoMsATG8u_UuTQ6hAHaE8.jpeg  
  inflating: raw-img/ragno/OIP-MZg0tznDRE95AMfyuutuYAHaE6.jpeg  
  inflating: raw-img/ragno/OIP-M_r9DkR2X5gP0P5-z8wPYQHaE7.jpeg  
  inflating: raw-img/ragno/OIP-M_xDUSzjlF3-w9XHCm93cgHaFF.jpeg  
  inflating: raw-img/ragno/OIP-MbI4-bMyutRX3X382IAVcQAAAA.jpeg  
  inflating: raw-img/ragno/OIP-MbMksZdD7gomsC9fd6cpKgHaHJ.jpeg  
  inflating: raw-img/ragno/OIP-Md4AMcvjaHcivJqeW06wEwHaFW.jpeg  
  inflating: raw-img/ragno/OIP-MgCgailkT5XkO1XczzQCngHaLK.jpeg  
  inflating: raw-img/ragno/OIP-MgrKq05YHGXsgj1AmYIyJQHaE8.jpeg  
  inflating: raw-img/ragno/OIP-MhRGx7d8nbnHBuqrsO00EQHaFo.jpeg  
  inflating: raw-img/ragno/OIP-MiVgMXcKGQdog2KGe7i0YQHaHh.jpeg  
  inflating: raw-img/ragno/OIP-Mis0lJf3Xbf3ulUg4DQ_EwHaFz.jpeg  
  inflating: raw-img/ragno/OIP-Mj15wALS

In [6]:
pip install split-folders tqdm

In [7]:
import split_folders
split_folders.fixed('raw-img', output="output", seed=1337, fixed=(100, 100), oversample=False)

Copying files: 26179 files [00:05, 4533.47 files/s]


In [8]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt





Using TensorFlow backend.


In [9]:
import tensorflow as tf
IMAGE_SIZE = [224, 224]

train_path = 'output/train'
valid_path = 'output/validation'

# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

# useful for getting number of classes
#folders = glob('output/train/*')

x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(10, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)


# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=[tf.keras.metrics.AUC()]
)
#model.add(Dense(60, input_dim=60, activation='relu', kernel_constraint=maxnorm(3)))
#model.add(Dropout(0.2))


58892288/58889256 [==============================] - 6s 0us/step


In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('output/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('output/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 24179 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [13]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
756/756 [==============================] - 393s 519ms/step - loss: 0.7956 - auc: 0.9370 - val_loss: 1.5484 - val_auc: 0.9622
Epoch 2/5
756/756 [==============================] - 379s 501ms/step - loss: 0.5538 - auc: 0.9681 - val_loss: 0.8450 - val_auc: 0.9713
Epoch 3/5
756/756 [==============================] - 381s 503ms/step - loss: 0.4901 - auc: 0.9739 - val_loss: 0.3739 - val_auc: 0.9753
Epoch 4/5
756/756 [==============================] - 372s 492ms/step - loss: 0.4492 - auc: 0.9766 - val_loss: 2.1618 - val_auc: 0.9776
Epoch 5/5
756/756 [==============================] - 372s 493ms/step - loss: 0.4235 - auc: 0.9785 - val_loss: 0.0642 - val_auc: 0.9792


In [14]:

testt_datagen = ImageDataGenerator(rescale = 1./255)
testt_set = testt_datagen.flow_from_directory('output/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
p=model.predict(testt_set)
for i in p :
  print(i)

  

Found 1000 images belonging to 10 classes.
[7.2664196e-05 5.3603593e-09 1.4081546e-07 1.6061473e-09 1.5443424e-04
 4.9296021e-09 6.5961501e-08 2.1592127e-03 6.7161331e-08 9.9761331e-01]
[3.49960894e-10 1.01312986e-10 2.44238088e-13 5.21639398e-09
 6.33349639e-09 8.54986498e-11 3.87519211e-10 6.31072726e-06
 2.47439110e-14 9.99993682e-01]
[2.5176076e-04 4.9735222e-04 1.7531350e-01 4.6599058e-10 1.4272776e-08
 2.8004074e-08 4.4138920e-01 3.8174972e-01 2.0244209e-10 7.9837220e-04]
[5.9873721e-04 9.8273015e-01 6.3034589e-10 1.5291803e-14 2.6219515e-09
 2.3519275e-11 1.5585541e-02 1.0851201e-03 3.4590390e-11 4.6178772e-07]
[1.1263362e-11 5.4501231e-10 4.8395853e-11 4.6919219e-11 9.9997997e-01
 2.6742236e-18 3.8031381e-10 2.0072204e-05 2.4009354e-13 1.6464549e-10]
[7.3422742e-01 2.4553528e-01 4.7732246e-04 4.0813152e-06 1.4755891e-03
 2.3579236e-07 9.7657098e-03 4.3629375e-03 2.3861287e-06 4.1490849e-03]
[1.0241407e-10 1.7980625e-08 1.3080002e-11 2.7916665e-21 2.5688982e-16
 5.0999437e-17 1.

In [32]:
import keras
testt_datagen = ImageDataGenerator(rescale = 1./255)
testt_set = testt_datagen.flow_from_directory('output/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

predictions =model.evaluate(testt_set)




#predictions = model.predict_generator(testt_set)
#predicted_classes = keras.np_utils.probas_to_classes(predictions)

#predicted_classes = convert_to_class(predictions)

Found 1000 images belonging to 10 classes.
32/32 [==============================] - 9s 283ms/step


In [33]:
predictions
#print('Test loss:', score[0]) 
#print('Test accuracy:', score[1]

[0.2597709596157074, 0.9787604808807373]

In [34]:
print(model.metrics_names)

['loss', 'auc']
